In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import math
import scipy
import copy
import os
import matplotlib.pyplot as plt
import bluranalysis as analysis
from libwallerlab.utilities import noise
import llops as yp
from libwallerlab.projects.motiondeblur import blurkernel
import libwallerlab.utilities.simulation as sim
import libwallerlab.utilities.noise as noise
# plt.style.use('deblur')

## Define Constants

In [5]:
# Figure output directory
figure_directory = os.path.expanduser('/Users/zfphil/Dropbox/Berkeley/Phase-Deblurring/fom2019/')

# Define arguments struct for functions
system_params = {'pixel_count': (2580, 2180),
                 'numerical_aperture': 0.25,
                 'magnification': 10,
                 'pixel_size': 6.5e-6,
                 'readout_noise': 20,
                 'dark_current': 2.5,
                 'illumination_min_pulse_time': 10e-6, # s
                 'motion_settle_time': 0.25, # seconds
                 'motion_acceleration': 1e4, # mm / s / s
                 'motion_velocity_max': 40, # mm / s
                 'illumination_beta': 0.5,
                 'motion_axis': 1,
                 'illuminance': 1000, # lux
                 'n_tests': 100,
                 'camera_is_color': False,
                 'camera_readout_time': 0.016, # seconds
                 'camera_quantum_efficency': 0.6,
                 'camera_ad_conversion': 0.46
                }

pulse_count = 25

## Calculate SNR Improvement

In [12]:
t_strobe = 1e-5
t_coded = t_strobe * pulse_count
dnf_coded = analysis.getOptimalDnf(pulse_count*2)
system_params_illum_copy =copy.deepcopy(system_params)
system_params_illum_copy['camera_ad_conversion'] = 0.46
system_params_illum_copy['camera_quantum_efficency'] = 0.9
system_params_illum_copy['illuminance'] = 1000
system_params_illum_copy['readout_noise'] = 10
system_params_illum_copy['dark_current'] = 0.5
system_params_illum_copy['velocity_max'] = 2
snr_coded = analysis.exposureTimeToSnr(t_coded, dnf=dnf_coded, **system_params_illum_copy)
snr_strobe = analysis.exposureTimeToSnr(t_strobe, dnf=1, **system_params_illum_copy)
# print(snr_coded / snr_strobe)
print(snr_coded)

0.44178658727445363


In [53]:
illuminance_list = 10000, 2000, 10000
readout_noise_list = 10, 20, 40
frame_rate = 3

# Initialize lists
snr_strobed_list = []
snr_sns_list = []
snr_coded_list = []
snr_coded_raw_list = []

for illuminance, readout_noise in zip(illuminance_list, readout_noise_list):   
    # Define illuminance
    system_params_illum = copy.deepcopy(system_params)
    system_params_illum['illuminance'] = illuminance
    system_params_illum['readout_noise'] = readout_noise

    # SNS
    t_sns, dnf_sns = analysis.frameRateToExposure(frame_rate, 'stop_and_stare', pulse_count=pulse_count, **system_params_illum)
    snr_sns = analysis.exposureTimeToSnr(t_sns, dnf=dnf_sns, **system_params_illum)
    snr_sns_list.append(snr_sns)

    # Strobed
    t_strobe, dnf_strobe = analysis.frameRateToExposure(frame_rate, 'strobe', pulse_count=pulse_count, **system_params_illum)
    snr_strobe = analysis.exposureTimeToSnr(t_strobe, dnf=dnf_strobe, **system_params_illum)
    snr_strobed_list.append(snr_strobe)

    # Coded
    t_coded, dnf_coded = analysis.frameRateToExposure(frame_rate, 'code', pulse_count=pulse_count, **system_params_illum)
    snr_coded_list.append(analysis.exposureTimeToSnr(t_coded, dnf=dnf_coded, **system_params_illum))
    snr_coded_raw_list.append( analysis.exposureTimeToSnr(t_coded, dnf=1, **system_params_illum))

print(snr_strobed_list)
print(snr_coded_list)
print(snr_coded_list[1] / snr_strobed_list[1])
print(snr_coded_list[2] / snr_strobed_list[2])

[12.087313939334086, 2.0365261845292437, 5.030824234223193]
[7.6618376086750954, 3.123811570884849, 7.008837882398209]
1.5338921711958928
1.3931788422897349


## Generate Blurry and Strobed Groups of Images as Examples
https://www.ptgrey.com/support/downloads/10501

In [54]:
# Generate object
object_true = np.abs(yp.pad(sim.ucb(shape=(400,400)), (512,512), center=True, pad_value='edge')) * 100

# Generater Blur kernel
vector, dnf = blurkernel.generateVector(n_pulses=30)
kernel = blurkernel.fromVector(vector, object_true.shape)

# Define SNR function
snr = lambda x: noise.snr(x, signal_roi=yp.Roi(shape=(40, 40), start=(280, 30)), noise_roi=yp.Roi(shape=(10, 10), start=(40, 40)))

# Generate blurry object
object_blurry = yp.convolve(object_true, kernel)

# Define SNR to generate images from
measurement_list = []
for snr_strobed, snr_coded, snr_coded_raw in zip(snr_strobed_list, snr_coded_list, snr_coded_raw_list):
    data_sublist = []
    
    # Generate strobed data
    data_sublist.append(noise.add(object_true, snr=snr_strobed))
    
    # Generate coded measurement
    data_sublist.append(noise.add(object_blurry, snr=snr_coded_raw))
    
    # Deconvolve blurry measurement
    data_sublist.append(yp.deconvolve(data_sublist[-1], kernel, reg=1e-3))
    
    # Append to measurement list
    measurement_list.append(data_sublist)
    

plt.figure(figsize=(12, 6))

index = 0
clim=(50,150)
cmap = 'gray'
plt.subplot(331)
plt.imshow(measurement_list[index][0], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][0]), snr_strobed_list[index]))
plt.clim(clim)
plt.axis('off')
plt.subplot(332)
plt.imshow(measurement_list[index][1], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][1]), snr_coded_raw_list[index]))
plt.clim(clim)
plt.axis('off')
plt.subplot(333)
plt.imshow(measurement_list[index][2], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][2]), snr_coded_list[index]))
plt.clim(clim)
plt.axis('off')

index = 1
plt.subplot(334)
plt.imshow(measurement_list[index][0], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][0]), snr_strobed_list[index]))
plt.clim(clim)
plt.axis('off')
plt.subplot(335)
plt.imshow(measurement_list[index][1], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][1]), snr_coded_raw_list[index]))
plt.clim(clim)
plt.axis('off')
plt.subplot(336)
plt.imshow(measurement_list[index][2], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][2]), snr_coded_list[index]))
plt.clim(clim)
plt.axis('off')

index = 2
plt.subplot(337)
plt.imshow(measurement_list[index][0], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][0]), snr_strobed_list[index]))
plt.clim(clim)
plt.axis('off')
plt.subplot(338)
plt.imshow(measurement_list[index][1], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][1]), snr_coded_raw_list[index]))
plt.clim(clim)
plt.axis('off')
plt.subplot(339)
plt.imshow(measurement_list[index][2], cmap=cmap)
plt.title('%g / %g' % (snr(measurement_list[index][2]), snr_coded_list[index]))
plt.clim(clim)
plt.axis('off')

plt.tight_layout()

<IPython.core.display.Javascript object>

In [ ]:
snr_coded_list

## Sweep System Pulse Count
The parameter pulse_count is a scaler which represents the ratio of the amount of pulses used vs the amount of pulses which would saturate the camera

In [ ]:
# Set frame rate
frame_rate = 10

# Stop and stare signal-to-noise
t_sns, dnf_sns = analysis.frameRateToExposure(frame_rate, 'stop_and_stare', **system_params)
snr_sns = analysis.exposureTimeToSnr(t_sns, dnf=dnf_sns, **system_params)
counts_sns, noise_dependent, noise_independent = analysis.exposureTimeToNoiseComponents(t_sns, dnf=dnf_sns, **system_params)
noise_independent = 1 if noise_independent == 0 else noise_independent
print("SNS illumination at %d fps will have exposure %g seconds, %g counts, %g SNR (dnf = %g), and a noise ratio of %g" % (frame_rate, t_sns, counts_sns, snr_sns, dnf_sns, noise_dependent / noise_independent))

# Strobed signal-to-noise
t_strobe, dnf_strobe = analysis.frameRateToExposure(frame_rate, 'strobe', **system_params)
snr_strobe = analysis.exposureTimeToSnr(t_strobe, dnf=dnf_strobe, **system_params)
counts_strobe, noise_dependent, noise_independent = analysis.exposureTimeToNoiseComponents(t_strobe, dnf=dnf_strobe, **system_params)
noise_independent = 1 if noise_independent == 0 else noise_independent
print("Strobed illumination at %d fps will have exposure %g seconds, %g counts, %g SNR (dnf = %g), and a noise ratio of %g" % (frame_rate, t_strobe, counts_strobe, snr_strobe, dnf_strobe, noise_dependent / noise_independent))

# Loop over illumination beta
for pulse_count in np.arange(1,1000,10):
    t_coded, dnf_coded = analysis.frameRateToExposure(frame_rate, 'code', pulse_count=pulse_count, **system_params)
    snr_coded = analysis.exposureTimeToSnr(t_coded, dnf=dnf_coded, **system_params)
    counts_coded, noise_dependent, noise_independent = analysis.exposureTimeToNoiseComponents(t_coded, dnf=dnf_coded, **system_params)
    noise_independent = 1 if noise_independent == 0 else noise_independent
    print("pulse_count=%g coded illumination at %d fps will have exposure %g seconds, %g counts, %g SNR (dnf = %g), and a noise ratio of %g" % (pulse_count, frame_rate, t_coded, counts_coded, snr_coded, dnf_coded, noise_dependent / noise_independent))

## Plot SNR vs Frame Rate for 10 Lux

In [ ]:
# Define which frame rates to use
frame_rates = np.arange(0.1, 50, 1)

# Define number of pulses for coded
pulse_count = 30

# Define which illuminance to use
illuminance_list =  [1000, 40000]

# Initialize lists
snr_strobe_list = []
snr_sns_list = []
snr_coded_list = [] # list of lists

# Loop over frame rates
for index, frame_rate in enumerate(frame_rates):
        
    # Loop over illuminance
    snr_sns_sublist, snr_strobed_sublist, snr_coded_sublist = [], [], []
    for illuminance in illuminance_list:
        
        # Define illuminance
        system_params_illum = copy.deepcopy(system_params)
        system_params_illum['illuminance'] = illuminance
        
        # SNS
        t_sns, dnf_sns = analysis.frameRateToExposure(frame_rate, 'stop_and_stare', pulse_count=pulse_count, **system_params_illum)
        snr_sns = analysis.exposureTimeToSnr(t_sns, dnf=dnf_sns, **system_params_illum)
        snr_sns_sublist.append(snr_sns)
        
        # Strobed
        t_strobe, dnf_strobe = analysis.frameRateToExposure(frame_rate, 'strobe', pulse_count=pulse_count, **system_params_illum)
        snr_strobe = analysis.exposureTimeToSnr(t_strobe, dnf=dnf_strobe, **system_params_illum)
        snr_strobed_sublist.append(snr_strobe)
    
        # Coded
        t_coded, dnf_coded = analysis.frameRateToExposure(frame_rate, 'code', pulse_count=pulse_count, **system_params_illum)
        snr_coded = analysis.exposureTimeToSnr(t_coded, dnf=dnf_coded, **system_params_illum)
        snr_coded_sublist.append(snr_coded)
        
    # Append
    snr_sns_list.append(snr_sns_sublist)
    snr_strobe_list.append(snr_strobed_sublist)
    snr_coded_list.append(snr_coded_sublist)
    
snr_sns_list_transpose = np.asarray(snr_sns_list).T.tolist()
snr_strobe_list_transpose = np.asarray(snr_strobe_list).T.tolist()
snr_coded_list_transpose = np.asarray(snr_coded_list).T.tolist()

# Perform plotting
lw = 3
plt.figure(figsize=(8, 8))

# Loop over illuminance
c = ['g', 'r', 'b', 'y']
for index, (illuminance, snr_sns, snr_strobe, snr_coded) in enumerate(zip(illuminance_list, snr_sns_list_transpose, snr_strobe_list_transpose, snr_coded_list_transpose)):
    plt.semilogy(frame_rates, snr_coded, linewidth=lw, label='Coded (%d lux)' % illuminance, c=c[index])
#     plt.semilogy(frame_rates, snr_sns, 'r-', linewidth=lw, label='Stop and Stare  (%d lux)' % illuminance)

    if index == 0:
        plt.semilogy(frame_rates, snr_strobe, 'k-', linewidth=lw, label='Stobed  (%d lux)' % illuminance)
    else:
        plt.semilogy(frame_rates, snr_strobe, 'k--', linewidth=lw, label='Stobed  (%d lux)' % illuminance)
    
# Configure figure
plt.legend()
plt.xlabel('Frame Rate (Hz)', fontsize=28)
plt.ylabel('Imaging SNR', fontsize=28)
plt.ylim((1e-1, 1000))
plt.xlim((0, 50))
plt.grid('on', which='both')
plt.title('Illuminance and SNR Improvement')

# Set up ticks
ax = plt.gca()
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(20) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(20)

# Save
plt.tight_layout()
# plt.savefig(os.path.join(figure_directory, 'snr_plot.png'))

## Generate Example Images

In [ ]:
import libwallerlab.utilities.simulation as sim
import libwallerlab.utilities.noise as noise

index = 100

snr_coded_0 = snr_coded_list_transpose[0][index]
snr_coded_1 = snr_coded_list_transpose[1][index]
snr_strobe_0 = snr_strobe_list_transpose[0][index]
snr_strobe_1 = snr_strobe_list_transpose[1][index]

# Generate object
object_true = np.abs(sim.ucb())

# Define SNR to generate images from
snr_list = [snr_coded_0, snr_strobe_0, snr_coded_1, snr_strobe_1]

noisy_object_list = []
for snr in snr_list:
    noisy_object_list.append(noise.add(object_true, snr=snr))
    
# Show result

plt.figure(figsize=(2,6))
clim = [0.5, 1.5]
for index, (noisy, snr) in enumerate(zip(noisy_object_list, snr_list)):
    plt.subplot(411 + index)
    plt.imshow(np.abs(noisy))
#     plt.title('SNR: %g' % snr)
    plt.clim(clim)
    plt.axis('off')
    
plt.savefig(os.path.join(figure_directory, 'snr_images.png'))